In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import logging
import datetime

In [56]:
def create_logger():
    logger_ = logging.getLogger('main')
    logger_.setLevel(logging.DEBUG)
    fh = logging.FileHandler('simple_lightgbm.log')
    fh.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = logging.Formatter('[%(levelname)s]%(asctime)s:%(name)s:%(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger_.addHandler(fh)
    logger_.addHandler(ch)


def get_logger():
    return logging.getLogger('main')


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    if len(np.unique(y_true)) == 14:
        classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
        class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
#     if len(np.unique(y_true)) > 14:
#         classes.append(99)
#         class_weight[99] = 2
    
    # Galaxy Case
    if len(np.unique(y_true)) == 5:
        classes = [6, 16, 53, 65, 92]
        class_weight = {6: 1, 16: 1, 53: 1, 65: 1, 92: 1}
        
    # Out of Galaxy Case
    if len(np.unique(y_true)) == 9:
        classes = [15, 42, 52, 62, 64, 67, 88, 90, 95]
        class_weight = {15: 2, 42: 1, 52: 1, 62: 1, 64: 2, 67: 1, 88: 1, 90: 1, 95: 1}
        
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False


def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    if len(np.unique(y_true)) == 14:
        classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
        class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
#     if len(np.unique(y_true)) > 14:
#         classes.append(99)
#         class_weight[99] = 2
        
    # Galaxy Case
    if len(np.unique(y_true)) == 5:
        classes = [6, 16, 53, 65, 92]
        class_weight = {6: 1, 16: 1, 53: 1, 65: 1, 92: 1}
        
    # Out of Galaxy Case
    if len(np.unique(y_true)) == 9:
        classes = [15, 42, 52, 62, 64, 67, 88, 90, 95]
        class_weight = {15: 2, 42: 1, 52: 1, 62: 1, 64: 2, 67: 1, 88: 1, 90: 1, 95: 1}    
        
        
        
        
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def predict_chunk(df_, clfs_, meta_, features, train_mean):

    df_['flux_ratio_sq'] = np.power(df_['flux'] / df_['flux_err'], 2.0)
    df_['flux_by_flux_ratio_sq'] = df_['flux'] * df_['flux_ratio_sq']

    # Group by object id
    aggs = get_aggregations()

    aggs = get_aggregations()
    aggs['flux_ratio_sq'] = ['sum']
    aggs['flux_by_flux_ratio_sq'] = ['sum']

    new_columns = get_new_columns(aggs)

    agg_ = df_.groupby('object_id').agg(aggs)
    agg_.columns = new_columns

    agg_ = add_features_to_agg(df=agg_)

    # Merge with meta data
    full_test = agg_.reset_index().merge(
        right=meta_,
        how='left',
        on='object_id'
    )

    full_test = full_test.fillna(train_mean)
    # Make predictions
    preds_ = None
    for clf in clfs_:
        if preds_ is None:
            preds_ = clf.predict_proba(full_test[features]) / len(clfs_)
        else:
            preds_ += clf.predict_proba(full_test[features]) / len(clfs_)

    # Compute preds_99 as the proba of class not being any of the others
    # preds_99 = 0.1 gives 1.769
    preds_99 = np.ones(preds_.shape[0])
    for i in range(preds_.shape[1]):
        preds_99 *= (1 - preds_[:, i])

    # Create DataFrame from predictions
    preds_df_ = pd.DataFrame(preds_, columns=['class_' + str(s) for s in clfs_[0].classes_])
    preds_df_['object_id'] = full_test['object_id']
    preds_df_['class_99'] = 0.14 * preds_99 / np.mean(preds_99) 

    print(preds_df_['class_99'].mean())

    del agg_, full_test, preds_
    gc.collect()

    return preds_df_


def save_importances(importances_):
    mean_gain = importances_[['gain', 'feature']].groupby('feature').mean()
    importances_['mean_gain'] = importances_['feature'].map(mean_gain['gain'])
    plt.figure(figsize=(8, 24))
    sns.barplot(x='gain', y='feature', data=importances_.sort_values('mean_gain', ascending=False))
    plt.tight_layout()
    plt.savefig(r'../feat/importances_{}.png'.format(datetime.datetime.now().strftime('%m%d_%H%M')))
    importances_.sort_values('mean_gain', ascending=False).to_csv(r'../feat/feat_rank_{}.csv'.format(datetime.datetime.now().strftime('%m%d_%H%M')), index=False)


def train_classifiers(full_train=None, y=None):

    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class': 14,
        'metric': 'multi_logloss',
        'learning_rate': 0.03,
        'subsample': .9,
        'colsample_bytree': .7,
        'reg_alpha': .01,
        'reg_lambda': .01,
        'min_split_gain': 0.01,
        'min_child_weight': 10,
        'n_estimators': 1000,
        'silent': -1,
        'verbose': -1,
        'max_depth': 3
    }
    oof_preds = np.zeros((len(full_train), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = full_train.iloc[trn_], y.iloc[trn_]
        val_x, val_y = full_train.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=100,
            early_stopping_rounds=50
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        get_logger().info(multi_weighted_logloss(val_y, clf.predict_proba(val_x, num_iteration=clf.best_iteration_)))

        imp_df = pd.DataFrame()
        imp_df['feature'] = full_train.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)

    get_logger().info('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))

    return clfs, importances, oof_preds


def get_aggregations():
    return {
        'mjd': ['min', 'max', 'size'],
        'passband': ['mean', 'std', 'var'],  # ''min', 'max', 'mean', 'median', 'std'],
#         'flux': ['min', 'max', 'mean', 'median', 'std'],
        'flux': ['min', 'max', 'mean', 'median', 'std', 'skew'],
        'flux_err': ['min', 'max', 'mean', 'median', 'std'],
        'detected': ['mean'],  # ''min', 'max', 'mean', 'median', 'std'],
    }


def get_new_columns(aggs):
    return [k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


def add_features_to_agg(df):
    df['mjd_diff'] = df['mjd_max'] - df['mjd_min']
    df['flux_diff'] = df['flux_max'] - df['flux_min']
    df['flux_dif2'] = (df['flux_max'] - df['flux_min']) / df['flux_mean']
    df['flux_w_mean'] = df['flux_by_flux_ratio_sq_sum'] / df['flux_ratio_sq_sum']
    df['flux_dif3'] = (df['flux_max'] - df['flux_min']) / df['flux_w_mean']



    del df['mjd_max'], df['mjd_min']

    return df

In [57]:
train = pd.read_csv('../input/training_set.csv')
train.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [68]:
train.columns

Index(['object_id', 'mjd', 'passband', 'flux', 'flux_err', 'detected'], dtype='object')

In [58]:
train.mjd.describe()

count    1.421705e+06
mean     6.017921e+04
std      3.092379e+02
min      5.958003e+04
25%      5.989905e+04
50%      6.019331e+04
75%      6.048722e+04
max      6.067436e+04
Name: mjd, dtype: float64

In [59]:
def get_time_endpoints(df):
    t_min = df.mjd.min()
    t_max = df.mjd.max()
    print('Min and Max MJD time : {}, {}'.format(t_min, t_max))
    return t_min, t_max

def add_feats_within_time_interval(int_n, t_min, t_max, df, db):
    print('Number of Intervals :', int_n)
    int_dur = (t_max - t_min)/int_n
    for i in range(int_n):
        
        db_fil = db[(db.mjd>=(t_min+i*int_dur))&(db.mjd<(t_min + (i+1)*int_dur))][['object_id','flux','passband']]
        print('Interval #{}, record quantity: {}'.format(i+1, db_fil.shape[0]))
        
        # interval_#_flux_？
        stats = db_fil.groupby('object_id', as_index=False)['flux'].agg({'interval_{}_flux_mean'.format(i+1):'mean',
                                                                          'interval_{}_flux_std'.format(i+1):'std',
                                                                          'interval_{}_flux_min'.format(i+1):'min',
                                                                          'interval_{}_flux_max'.format(i+1):'max',
                                                                         'interval_{}_flux_skew'.format(i+1):'skew'})
        print('New features added: ',stats.columns.tolist())
        df = df.merge(stats, on='object_id', how='left')
        
        
        # interval_#_band_#_flux_？
#         stats = db_fil.groupby(['object_id','passband'])['flux'].skew().unstack()
#         stats.columns = ['interval_{}_band_{}_flux_skew'.format(i+1, str(col)) for col in stats.columns.tolist()]
#         print('Feats added:',stats.columns.tolist())
#         stats['object_id'] = stats.index    
#         df = df.merge(stats, on='object_id', how='left').fillna(0) 
        
                
    # interval_#_flux_？ 互相做差
#     for key in ['max', 'min', 'mean']:
#     for key in ['max']:
#         key_cols = ['interval_{}_flux_{}'.format(i, key) for i in range(1, int_n+1)]
#         for col in key_cols:
#             subtract_cols = [col_ for col_ in key_cols if col_ < col]
#             for sub_col in subtract_cols:
#                 df['{}_minus_{}'.format(col, sub_col)] = df[col] - df[sub_col]
#                 print('Feature added:', '{}_minus_{}'.format(col, sub_col))
        

    
    return df

In [60]:
meta_train = pd.read_csv('../input/training_set_metadata.csv')
meta_train.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


In [61]:
meta_train[~meta_train.distmod.isnull()].target.nunique()

9

In [62]:
def add_band_feats(df, db):
    
    
### 均值
    print('Adding feats for the flux mean per band...')
    stats = db.groupby(['object_id','passband'])['flux'].mean().unstack()
    stats.columns = ['band_' + str(col) + '_flux_mean' for col in stats.columns.tolist()]
    
    # band_#_flux_mean互相减去
    mean_cols = stats.columns.tolist()
    for col in mean_cols:
        subtract_cols = [col_ for col_ in mean_cols if col_ < col]
        for sub_col in subtract_cols:
            stats['{}_minus_{}'.format(col, sub_col)] = stats[col] - stats[sub_col]
      
    print('Feats added:',stats.columns.tolist())
    stats['object_id'] = stats.index    
    df = df.merge(stats, on='object_id', how='left').fillna(0)
    
    
### 标准差    
    print('Adding feats for the flux std per band...')
    stats = db.groupby(['object_id','passband'])['flux'].std().unstack()
    stats.columns = ['band_' + str(col) + '_flux_std' for col in stats.columns.tolist()]
    print('Feats added:',stats.columns.tolist())
    stats['object_id'] = stats.index    
    df = df.merge(stats, on='object_id', how='left').fillna(0)

    
### 偏度    
    print('Adding feats for the flux skew per band...')
    stats = db.groupby(['object_id','passband'])['flux'].skew().unstack()
    stats.columns = ['band_' + str(col) + '_flux_skew' for col in stats.columns.tolist()]
    print('Feats added:',stats.columns.tolist())
    stats['object_id'] = stats.index    
    df = df.merge(stats, on='object_id', how='left').fillna(0) 
    

### 最大值
    print('Adding feats for the flux max per band...')
    stats = db.groupby(['object_id','passband'])['flux'].max().unstack()
    stats.columns = ['band_' + str(col) + '_flux_max' for col in stats.columns.tolist()]
    # band_#_flux_max互相减去
    max_cols = stats.columns.tolist()
    for col in max_cols:
        subtract_cols = [col_ for col_ in max_cols if col_ < col]
        for sub_col in subtract_cols:
            stats['{}_minus_{}'.format(col, sub_col)] = stats[col] - stats[sub_col]
            
    print('Feats added:',stats.columns.tolist())
    stats['object_id'] = stats.index    
    df = df.merge(stats, on='object_id', how='left').fillna(0) 
    

    
### 最小值    
    print('Adding feats for the flux min per band...')
    stats = db.groupby(['object_id','passband'])['flux'].min().unstack()
    stats.columns = ['band_' + str(col) + '_flux_min' for col in stats.columns.tolist()]
    # band_#_flux_min互相做差
    min_cols = stats.columns.tolist()
    for col in min_cols:
        subtract_cols = [col_ for col_ in min_cols if col_ < col]
        for sub_col in subtract_cols:
            stats['{}_minus_{}'.format(col, sub_col)] = stats[col] - stats[sub_col]
    print('Feats added:',stats.columns.tolist())
    stats['object_id'] = stats.index    
    df = df.merge(stats, on='object_id', how='left').fillna(0) 
    

    
    
# ### ......  Mean
#     print('Adding feats for the flux_err mean per band...')
#     stats = db.groupby(['object_id','passband'])['flux_err'].mean().unstack()
#     stats.columns = ['band_' + str(col) + '_flux_err_mean' for col in stats.columns.tolist()]      
#     print('Feats added:',stats.columns.tolist())
#     stats['object_id'] = stats.index    
#     df = df.merge(stats, on='object_id', how='left').fillna(0)
    
    
    
    
### 遍历band_list计算 
    print('Adding feats for the flux (max-min)/mean per band...')
    for band_n in range(6):
        df['band_' + str(band_n) + '_flux_diff1'] = df['band_' + str(band_n) + '_flux_max'] - df['band_' + str(band_n) + '_flux_min']
        df['band_' + str(band_n) + '_flux_diff2'] = df['band_' + str(band_n) + '_flux_diff1']/df['band_' + str(band_n) + '_flux_mean']
        print('Feature added: band_' + str(band_n) + '_flux_diff2')
        
#         df['band_' + str(band_n) + '_flux_err_ratio'] = df['band_' + str(band_n) + '_flux_err_mean']/df['band_' + str(band_n) + '_flux_mean']
#         print('Feature added: band_' + str(band_n) + '_flux_err_ratio')
        
        
        

    


    
    return df

    
#     print('Adding feats for the flux mean per band...')
#     stats = db.groupby(['object_id','passband'])['flux'].mean().unstack()
#     stats['object_id'] = stats.index
#     stats.columns = [str(col) + '_mean' for col in stats.columns.tolist()]
#     df = df.merge(db, on='object_id', how='left').fillna(0)
#     print('Feats added:',stats.columns.tolist())
    
    
    

In [63]:
# def add_ratio_col(df):
#     df['hostgal_photoz_ratio'] = df['hostgal_photoz_err']/df['hostgal_photoz']
#     df['hostgal_photoz_ratio'].fillna(0, inplace=True)
    
#     return df

In [71]:
def main():
    train = pd.read_csv('../input/training_set.csv')
    
    # 增加按MJD划分时间统计得到的特征
    mjd_min, mjd_max = get_time_endpoints(train)
    # 增加按MJD划分时间统计得到的特征
    
    
    train['flux_ratio_sq'] = np.power(train['flux'] / train['flux_err'], 2.0)
    train['flux_by_flux_ratio_sq'] = train['flux'] * train['flux_ratio_sq']


    aggs = get_aggregations()
    aggs['flux_ratio_sq'] = ['sum']
    aggs['flux_by_flux_ratio_sq'] = ['sum']

    
    agg_train = train.groupby('object_id').agg(aggs)
    new_columns = get_new_columns(aggs)
    agg_train.columns = new_columns

    agg_train = add_features_to_agg(df=agg_train)

    meta_train = pd.read_csv('../input/training_set_metadata.csv')

    
    # 增加是否在银河系的特征
    meta_train.distmod.fillna(0,inplace=True)
    meta_train['in_galaxy'] = 0
    meta_train.loc[(meta_train.distmod == 0), 'in_galaxy'] = 1
    # 增加是否在银河系的特征

    full_train = agg_train.reset_index().merge(
        right=meta_train,
        how='outer',
        on='object_id'
    )
    
    del full_train['hostgal_specz']
    
    
    

#     y = full_train['target']
#     del full_train['target']

      
    # TO DO!!!!! Seperately calculated within objects in galaxy
    train_mean = full_train.mean(axis=0)
    full_train.fillna(train_mean, inplace=True)
    

    
    # 增加 mean_flux_per_band
    full_train = add_band_feats(full_train, train)
    # 增加 mean_flux_per_band
    
    
    # 增加按MJD划分时间统计得到的特征
    full_train = add_feats_within_time_interval(6, mjd_min, mjd_max, full_train, train)
    # 增加按MJD划分时间统计得到的特征 
    
#     # 增加ratio特征
#     full_train = add_ratio_col(full_train)
#     # 增加ratio特征

    objects = full_train['object_id'].unique().tolist()
    print('Number of objects :',len(objects))
   
    full_train_in_gal = full_train[full_train['in_galaxy']==1]
    objects_in_gal = full_train_in_gal['object_id'].unique().tolist()
    print('Number of objects in galaxy :',len(objects_in_gal))
    
    full_train_out_gal = full_train[full_train['in_galaxy']==0]
    objects_out_gal = full_train_out_gal['object_id'].unique().tolist()
    print('Number of objects out of galaxy :',len(objects_out_gal))
    print('Just to check, sum of objects :', len(objects_in_gal) + len(objects_out_gal))
    
    del train
    
    y_list = []
    preds_list = []
    for df in [full_train_in_gal, full_train_out_gal]:
        del df['object_id']
        y = df['target']
        y_list.append(y)
        del df['target']
        gc.collect()
        get_logger().info(df.columns)
        clfs, importances, preds = train_classifiers(df, y)
        preds_list.append(preds)
    
    print('Let us check the logloss when objects are trained separately...')
    all_y = pd.concat(y_list, ignore_index=True)

    preds_in_gal = preds_list[0]
    preds_out_gal = preds_list[1]
    in_classes = [6, 16, 53, 65, 92]
    out_classes = [15, 42, 52, 62, 64, 67, 88, 90, 95]
    print(preds_in_gal)
    preds_in_gal[out_classes] = 0
    preds_out_gal[in_classes] = 0
    print(preds_in_gal)
    
    all_preds = np.concatenate((preds_in_gal, preds_out_gal), axis=0)
    
    
    

    
    
    
    
    
    
    get_logger().info('Separate MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=all_y, y_preds=all_preds))
    
    
    
        
    
    
    
    
#     del full_train['object_id']
        

#     gc.collect()

        
#     get_logger().info(full_train.columns)
#     clfs, importances = train_classifiers(full_train, y)

#     save_importances(importances_=importances)

#     meta_test = pd.read_csv('../input/test_set_metadata.csv')

#     import time

#     start = time.time()
#     chunks = 5000000
#     remain_df = None

#     for i_c, df in enumerate(pd.read_csv('../input/test_set.csv', chunksize=chunks, iterator=True)):
#         # Check object_ids
#         # I believe np.unique keeps the order of group_ids as they appear in the file
#         unique_ids = np.unique(df['object_id'])
#         new_remain_df = df.loc[df['object_id'] == unique_ids[-1]].copy()

#         if remain_df is None:
#             df = df.loc[df['object_id'].isin(unique_ids[:-1])].copy()
#         else:
#             df = pd.concat([remain_df, df.loc[df['object_id'].isin(unique_ids[:-1])]], axis=0)

#         # Create remaining samples df
#         remain_df = new_remain_df

#         preds_df = predict_chunk(df_=df,
#                                  clfs_=clfs,
#                                  meta_=meta_test,
#                                  features=full_train.columns,
#                                  train_mean=train_mean)

#         if i_c == 0:
#             preds_df.to_csv('predictions_v3.csv', header=True, index=False, float_format='%.6f')
#         else:
#             preds_df.to_csv('predictions_v3.csv', header=False, mode='a', index=False, float_format='%.6f')

#         del preds_df
#         gc.collect()

#         if (i_c + 1) % 10 == 0:
#             get_logger().info('%15d done in %5.1f' % (chunks * (i_c + 1), (time.time() - start) / 60))
#             print('%15d done in %5.1f' % (chunks * (i_c + 1), (time.time() - start) / 60))

#     # Compute last object in remain_df

#     preds_df = predict_chunk(df_=remain_df,
#                              clfs_=clfs,
#                              meta_=meta_test,
#                              features=full_train.columns,
#                              train_mean=train_mean)

#     preds_df.to_csv('predictions_v3.csv', header=False, mode='a', index=False, float_format='%.6f')

#     z = pd.read_csv('predictions_v3.csv')

#     z = z.groupby('object_id').mean()

#     z.to_csv('single_predictions_v3.csv', index=True, float_format='%.6f')




In [72]:
gc.enable()
create_logger()
try:
    main()
except Exception:
    get_logger().exception('Unexpected Exception Occured')
    raise

Min and Max MJD time : 59580.0343, 60674.363
Adding feats for the flux mean per band...
Feats added: ['band_0_flux_mean', 'band_1_flux_mean', 'band_2_flux_mean', 'band_3_flux_mean', 'band_4_flux_mean', 'band_5_flux_mean', 'band_1_flux_mean_minus_band_0_flux_mean', 'band_2_flux_mean_minus_band_0_flux_mean', 'band_2_flux_mean_minus_band_1_flux_mean', 'band_3_flux_mean_minus_band_0_flux_mean', 'band_3_flux_mean_minus_band_1_flux_mean', 'band_3_flux_mean_minus_band_2_flux_mean', 'band_4_flux_mean_minus_band_0_flux_mean', 'band_4_flux_mean_minus_band_1_flux_mean', 'band_4_flux_mean_minus_band_2_flux_mean', 'band_4_flux_mean_minus_band_3_flux_mean', 'band_5_flux_mean_minus_band_0_flux_mean', 'band_5_flux_mean_minus_band_1_flux_mean', 'band_5_flux_mean_minus_band_2_flux_mean', 'band_5_flux_mean_minus_band_3_flux_mean', 'band_5_flux_mean_minus_band_4_flux_mean']
Adding feats for the flux std per band...


C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: 'object_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: 'object_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


Feats added: ['band_0_flux_std', 'band_1_flux_std', 'band_2_flux_std', 'band_3_flux_std', 'band_4_flux_std', 'band_5_flux_std']
Adding feats for the flux skew per band...
Feats added: ['band_0_flux_skew', 'band_1_flux_skew', 'band_2_flux_skew', 'band_3_flux_skew', 'band_4_flux_skew', 'band_5_flux_skew']
Adding feats for the flux max per band...
Feats added: ['band_0_flux_max', 'band_1_flux_max', 'band_2_flux_max', 'band_3_flux_max', 'band_4_flux_max', 'band_5_flux_max', 'band_1_flux_max_minus_band_0_flux_max', 'band_2_flux_max_minus_band_0_flux_max', 'band_2_flux_max_minus_band_1_flux_max', 'band_3_flux_max_minus_band_0_flux_max', 'band_3_flux_max_minus_band_1_flux_max', 'band_3_flux_max_minus_band_2_flux_max', 'band_4_flux_max_minus_band_0_flux_max', 'band_4_flux_max_minus_band_1_flux_max', 'band_4_flux_max_minus_band_2_flux_max', 'band_4_flux_max_minus_band_3_flux_max', 'band_5_flux_max_minus_band_0_flux_max', 'band_5_flux_max_minus_band_1_flux_max', 'band_5_flux_max_minus_band_2_flu

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: 'object_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:52: FutureWarning: 'object_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


Adding feats for the flux min per band...
Feats added: ['band_0_flux_min', 'band_1_flux_min', 'band_2_flux_min', 'band_3_flux_min', 'band_4_flux_min', 'band_5_flux_min', 'band_1_flux_min_minus_band_0_flux_min', 'band_2_flux_min_minus_band_0_flux_min', 'band_2_flux_min_minus_band_1_flux_min', 'band_3_flux_min_minus_band_0_flux_min', 'band_3_flux_min_minus_band_1_flux_min', 'band_3_flux_min_minus_band_2_flux_min', 'band_4_flux_min_minus_band_0_flux_min', 'band_4_flux_min_minus_band_1_flux_min', 'band_4_flux_min_minus_band_2_flux_min', 'band_4_flux_min_minus_band_3_flux_min', 'band_5_flux_min_minus_band_0_flux_min', 'band_5_flux_min_minus_band_1_flux_min', 'band_5_flux_min_minus_band_2_flux_min', 'band_5_flux_min_minus_band_3_flux_min', 'band_5_flux_min_minus_band_4_flux_min']


...

[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199
[INFO]2018-10-26 17:46:16,327:main:1.2374936873479199


Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 0.9253	training's wloss: 1.15564	valid_1's multi_logloss: 1.02136	valid_1's wloss: 1.37753
[200]	training's multi_logloss: 0.725477	training's wloss: 0.912138	valid_1's multi_logloss: 0.893709	valid_1's wloss: 1.27375
[300]	training's multi_logloss: 0.620919	training's wloss: 0.748793	valid_1's multi_logloss: 0.854586	valid_1's wloss: 1.23211
[400]	training's multi_logloss: 0.546618	training's wloss: 0.629779	valid_1's multi_logloss: 0.839868	valid_1's wloss: 1.21994
[500]	training's multi_logloss: 0.48813	training's wloss: 0.542624	valid_1's multi_logloss: 0.831018	valid_1's wloss: 1.2166
Early stopping, best iteration is:
[512]	training's multi_logloss: 0.481882	training's wloss: 0.533461	valid_1's multi_logloss: 0.829983	valid_1's wloss: 1.2156


[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,804:main:1.2156008320234841
[INFO]2018-10-26 17:46:28,823:main:MULTI WEIGHTED LOG LOSS : 1.21493 
[INFO]2018-10-26 17:46:28,823:main:MULTI WEIGHTED LOG LOSS : 1.21493 
[INFO]2018-10-26 17:46:28,823:main:MULTI WEIGHTED LOG LOSS : 1.21493 
[INFO]2018-10-26 17:46:28,823:main

Let us check the logloss when objects are trained separately...
[[6.06056631e-04 8.98046626e-03 9.37368349e-04 5.17867431e-04
  9.88958241e-01]
 [2.59190612e-04 3.08585546e-04 2.31513767e-04 9.99075888e-01
  1.24822029e-04]
 [1.45169491e-04 7.48674133e-03 2.66576104e-04 9.91978526e-01
  1.22987430e-04]
 ...
 [3.69622977e-02 9.19472009e-01 7.39521507e-03 2.87742007e-02
  7.39627723e-03]
 [2.48841160e-03 2.81507799e-04 3.31812427e-04 9.96468606e-01
  4.29662438e-04]
 [9.63891255e-01 5.48330708e-04 2.87639271e-02 3.14884926e-03
  3.64763771e-03]]
[[6.06056631e-04 8.98046626e-03 9.37368349e-04 5.17867431e-04
  9.88958241e-01]
 [2.59190612e-04 3.08585546e-04 2.31513767e-04 9.99075888e-01
  1.24822029e-04]
 [1.45169491e-04 7.48674133e-03 2.66576104e-04 9.91978526e-01
  1.22987430e-04]
 ...
 [3.69622977e-02 9.19472009e-01 7.39521507e-03 2.87742007e-02
  7.39627723e-03]
 [2.48841160e-03 2.81507799e-04 3.31812427e-04 9.96468606e-01
  4.29662438e-04]
 [9.63891255e-01 5.48330708e-04 2.87639271e-0

ValueError: all the input array dimensions except for the concatenation axis must match exactly